# Particle Neural Network

Imports Three handcrafted files, in addition to the standard numpy and matplotlib moduals.  ParticleNet.py is the neural network, which takes as input 4-vectors generated using ExpodentialDistribution.py and GaussianDistribution.py which produced 3-vector momentum and a mass vector each, where the mass vector is radomly drawn from either an expodential or a gaussian distribution respectivly.  PlotResults contains a plot function, which plots the distribution of the precentages of the Particle Neural Network's predictions for a single particle.

In [1]:
from ExpodentialDistribution import Exp
from GaussianDistribution import Gauss
from ParticleNet import ParticleNet
import numpy as np
from PlotResults import plot
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
params = 4

These are the only constants needed to run this simulation thus far.  'p' the momentum is constant for all particles.  Ntrain is the number of samples the particle neural network will train on, and Ntest is the number of particles the neural network will test on.

In [2]:
p = 100
Ntrain = 5
Ntest = 5

Below, we generate our data to feed into the particle neural network.  First we create training and testing data sets from the expodential distribution modual and then the gaussian modual.  Both moduals generate a 4-vector, which includes a 3-vector momentum and a mass vector where the masses are drawn from the respective distribution at random.  Besides the 4-vector, a vector idicating that which distribution each piece of data belongs to is returned, a vector of all zeros for the expodential distribution and a vector of all ones for the gaussian.

In [3]:
ETrain,ETrainVals = Exp(p,Ntrain)
ETest,ETestVals = Exp(p,Ntest)

GTrain,GTrainVals = Gauss(p,Ntrain)
GTest,GTestVals = Gauss(p,Ntest)

Here, we are just concatinating the data from the expodential and gaussian distributions into a single training and testing data set.

In [4]:
Train = np.zeros((2*Ntrain,params+1))
Test = np.zeros((2*Ntest,params+1))

Train[:Ntrain,0:params] = ETrain
Train[Ntrain:,0:params] = GTrain
Train[:Ntrain,params] = ETrainVals
Train[Ntrain:,params] = GTrainVals

Test[:Ntest,0:params] = ETest
Test[Ntest:,0:params] = GTest
Test[:Ntrain,params] = ETrainVals
Test[Ntrain:,params] = GTrainVals

BUFFER_SIZE = Ntrain*2
BATCH_SIZE = params+1
train_dataset = tf.data.Dataset.from_tensor_slices(Train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(Test).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [5]:
for i in train_dataset.take(-1):
    train = i[0:,0:params]
    train_vals = i[0:,params]
for i in test_dataset.take(-1):
    test = i[0:,0:params]
    test_vals = i[0:,params]
train = np.array(train)
train_vals = np.array(train_vals)
test = np.array(test)
test_vals = np.array(test_vals)

In [258]:
def make_generator_model(params,N):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, use_bias=False, input_shape=(4,1)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())    
    
    model.add(layers.Reshape((params, 256,1)))
    assert model.output_shape == (None, params, 256,1)
    
#    for i in range(params-3):
#        x = (params*256)/((i+1)*2)
    model.add(layers.Conv2DTranspose(128, 1, strides=1, padding='same', use_bias=False))
    assert model.output_shape == (None, params, 128,1)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2DTranspose(1, 1, strides=1, padding='same', use_bias=False))
    assert model.output_shape == (None, params, 1, 1)
    
    return model

In [259]:
print(params*256./((0+1)*2))

512.0


In [260]:
generator = make_generator_model(params,Ntrain)

AssertionError: 

In [155]:
noise = tf.random.uniform([params])
generated_image = generator(noise, training=False)

ValueError: Input 0 of layer sequential_14 is incompatible with the layer: : expected min_ndim=2, found ndim=1. Full shape received: (4,)

Below is the particle neural network being run on the data generated.

In [ ]:
predict = ParticleNet(train,train_vals,test)

Below, we take a look at the masses for the expodential distribution.

In [ ]:
n, bins, patches = plt.hist(x=ETrain[:,3], bins='auto')
plt.xlabel('Masses Expodential')
plt.ylabel('Frequency')
plt.title('Expodential Distrubution')

Below is a plot of the masses randomly generated from a Gaussian distribution.

In [ ]:
n, bins, patches = plt.hist(x=GTrain[:,3], bins='auto')
plt.xlabel('Masses Normal')
plt.ylabel('Frequency')
plt.title('Normal Distrubution')

In [ ]:
n2, bins2, patches2 = plt.hist(x=ETrain[:,3], bins=np.arange(0,1000,5))
n, bins, patches = plt.hist(x=GTrain[:,3], bins=np.arange(0,1000,5))
plt.xlim(0,600)
plt.ylabel('Frequency')
plt.title('Normal Distrubution')

In [ ]:
integN = np.sum(n2)
integG = np.sum(n)
print(integN)
print(integG)
xmin = np.argwhere(bins==25).flatten()[0]
xmax = np.argwhere(bins==75).flatten()[0]
ratio = np.sum(n2[xmin:xmax])/np.sum(n[xmin:xmax])
print(ratio)

In [ ]:
FalseExps = sum(predict[:Ntest])/Ntest*100
print("The model, on average, thought a Expodential distribution to be just that at " + str(FalseExps[0]) + " percent.")
print("And thought a Expodential distribution to be a Gaussian distribution at " + str(FalseExps[1]) + " percent.")

In [ ]:
FalseGauss = (Ntest-sum(predict[Ntest:]))/Ntest*100
print("The model, on average, thought a Gaussian distribution to be just that at " + str(FalseGauss[0]) + " percent.")
print("And thought a Gaussian distribution to be a Expodential distribution at " + str(FalseGauss[1]) + " percent.")

In [ ]:
dist = Train[:,3]*predict[:,0]
plt.hist(x=Train[:,3],weights=predict[:,1],alpha=0.5, bins=np.arange(0,1000,5))
plt.hist(x=Train[:,3],weights=predict[:,0],alpha=0.5, bins=np.arange(0,1000,5))
plt.xlim(0,200)

In [ ]:
n2, bins2, patches2 = plt.hist(x=ETrain[:,3], bins=np.arange(0,1000,5))
n, bins, patches = plt.hist(x=GTrain[:,3], bins=np.arange(0,1000,5))
plt.xlim(0,600)
plt.ylabel('Frequency')